In [1]:
import gym
import collections

from torch.utils.tensorboard import SummaryWriter

In [2]:
GAMMA = 0.9
NUM_TEST_EPISODES = 20
GYM_ENV = "FrozenLake8x8-v0"

In [5]:
class Agent:
    def __init__(self):
        self.env = gym.make(GYM_ENV)
        self.state = self.env.reset()
        # {(state, action, new_state): reward}
        self.rewards = collections.defaultdict(float)
        # {(state, action): {tgt_state: count}}
        self.transitions = collections.defaultdict(collections.Counter)
        # {(state, action): value}
        self.values = collections.defaultdict(float)
        
    def value_iteration(self):
        """
        to build some knowledge about the env
        namely, saving the value of each action for each state
        implements Bellman approximation
        """
        for state in range(self.env.observation_space.n):
            for action in range(self.env.action_space.n):
                target_counts = self.transitions[(state, action)]
                total = sum(target_counts.values())
                action_value = 0
                for tgt_state, count in target_counts.items():
                    reward = self.rewards[(state, action, tgt_state)]
                    best_action = self.select_action(tgt_state)
                    val = reward + GAMMA * self.values[(tgt_state, best_action)]
                    action_value += (count/total) * val
                self.values[(state, action)] = action_value

    def play_n_random_steps(self, count):
        """
        to build some knowledge about the env
        namely, reward given for state + action -> new_state
        and (transitions,) the probability of state + action -> new_state (over all possible [new_state]s)
        """
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transitions[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state

    def play_episode(self, env):
        """
        use built knowledge on outside env
        """
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transitions[(state, action)] [new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
        
    def select_action(self, state):
        """
        determines 'best' action at a state by argmaxing all saved state-action values
        """
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

In [6]:
# train
test_env = gym.make(GYM_ENV)
agent = Agent()
writer = SummaryWriter(comment=f'-q-iteration-{GYM_ENV}')

i = 0
best_reward = 0.0
while True:
    i += 1
    agent.play_n_random_steps(100)
    agent.value_iteration()
    
    reward = 0.0
    for _ in range(NUM_TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= NUM_TEST_EPISODES
    writer.add_scalar("reward", reward, i)
    
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward

    if reward > 0.8:
        print("Solved in %d iterations" % i)
        break

writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.100
Best reward updated 0.100 -> 0.400
Best reward updated 0.400 -> 0.500
Best reward updated 0.500 -> 0.600
Best reward updated 0.600 -> 0.700
Best reward updated 0.700 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 1025 iterations
